In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

def print_input_files():
    import os
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install necessary modules

!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get update
!apt-get -y install python3-tflite-runtime
# !pip install https://dl.google.com/coral/python/tflite_runtime-2.1.0.post1-cp36-cp36m-linux_aarch64.whl

# !pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  69402      0 --:--:-- --:--:-- --:--:-- 69973
OK
Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5002 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6332 B]
Get:5 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [2356 B]
Get:6 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Ign:9 https

In [3]:
class Dataset:
    def __init__(self, train_file_path, base_folder_path):
        if base_folder_path.endswith("/"):
            self.base_folder_path = base_folder_path
        else:
            self.base_folder_path = base_foldder_path + "/"
        
        self.train_data = pd.read_csv(train_file_path)
        self.relevant_data = None
        self.pq_path = None
        
    def get_relevant_data_subset(self, pq_path, sequence_id):
        if pq_path == self.pq_path:
            return self.relevant_data.loc[sequence_id]
        
        self.relevant_data = pd.read_parquet(base_folder_path + pq_path)
        self.pq_path = pq_path
        return self.relevant_data.loc[sequence_id]

In [4]:
base_folder_path = "/kaggle/input/asl-fingerspelling/"
train_file_path = "/kaggle/input/asl-fingerspelling/train.csv"
dataset = Dataset(train_file_path, base_folder_path)

In [5]:
def calculate_frame_frequency():    
    total_frames = 0
    total_output_size = 0
    for pq_path, sequence_id, output in zip(dataset.train_data['path'], dataset.train_data['sequence_id'], dataset.train_data['phrase']):
        relevant_data = dataset.get_relevant_data_subset(pq_path, sequence_id)
        total_frames += len(relevant_data)
        total_output_size += len(output)

    print(total_frames)
    print(total_output_size)
    print("Average frames per character: {}".format(total_frames / total_output_size))

In [6]:
from collections import defaultdict, Counter

counter = Counter(''.join(dataset.train_data['phrase']))    
most_common_character = counter.most_common(1)[0][0]

In [7]:
# TODO
# 1. Load character map
# 2. Initialize weights so that it always map to most common character
# 3. Build model
# 4. Convert into TFLite
# 5. Submit

In [8]:
import json

with open ("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)
rev_character_map = {j:i for i,j in character_map.items()}

print(len(rev_character_map))

59


Apparently we are not given a video but rather 3D coordinates of the different points of the body, probably for hiding sensitive information.

In [9]:
import tensorflow as tf
from tensorflow.keras import models, layers

output_size = len(rev_character_map)
input_size = dataset.get_relevant_data_subset(dataset.train_data.iloc[0]['path'], dataset.train_data.iloc[0]['sequence_id']).shape[1]

model = models.Sequential()
layer = layers.Dense(output_size, input_shape=(input_size,))

layer.build((None, input_size))

weights = np.zeros((input_size, output_size))
bias = np.zeros((output_size,))
bias[character_map[most_common_character]] = 1

layer.set_weights([weights, bias])
layer.trainable = False

model.add(layer)
model.compile(optimizer='adam', loss='mse')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [11]:
!zip -r submission.zip model.tflite

  adding: model.tflite (deflated 100%)


In [12]:
# Equivalent of how the model is evaluated

def evaluate_model():
    model_path = "/kaggle/working/model.tflite"
    frames = dataset.get_relevant_data_subset(dataset.train_data.iloc[0]['path'], dataset.train_data.iloc[0]['sequence_id'])

    import tflite_runtime.interpreter as tflite
    interpreter = tflite.Interpreter(model_path)

    REQUIRED_SIGNATURE = "serving_default"
    REQUIRED_OUTPUT = "outputs"

    with open ("/kaggle/input/fingerspelling-character-map/character_to_prediction_index.json", "r") as f:
        character_map = json.load(f)
    rev_character_map = {j:i for i,j in character_map.items()}

    found_signatures = list(interpreter.get_signature_list().keys())

    if REQUIRED_SIGNATURE not in found_signatures:
        raise KernelEvalException('Required input signature not found.')

    prediction_fn = interpreter.get_signature_runner("serving_default")
    output = prediction_fn(inputs=frames)
    prediction_str = "".join([rev_character_map.get(s, "") for s in np.argmax(output[REQUIRED_OUTPUT], axis=1)])
    return prediction_str